In [1]:
import sys
path_to_pdn = r"\\t999\Сетевой диск\pdn_"
# сюда внутри кавычек вставляем путь, где ледит репозиторий пдн
sys.path.insert(0, path_to_pdn)

In [2]:
import os
import pandas as pd
import datetime as dt
import numpy as np
from pdn.download_data.download_calls_data import download_calls_data

In [3]:
# путь файла с текущими статусами пациентов
patient_current_statuses_path = r''
patient_current_statuses_path = patient_current_statuses_path.split('\\')
patient_current_statuses_path = os.path.join(r'\\', *patient_current_statuses_path)

In [4]:
# путь папки с данными по распределению помощников врачей по врачам
assistant_to_doctor_distribution_path = r''
assistant_to_doctor_distribution_path = assistant_to_doctor_distribution_path.split('\\')
assistant_to_doctor_distribution_path = os.path.join(r'\\', *assistant_to_doctor_distribution_path)

In [5]:
# путь папки с данными по звонкам
calls_data_path = r'\\t999\Сетевой диск\pdn\data\from_call_program'
calls_data_path = calls_data_path.split('\\')
calls_data_path = os.path.join(r'\\', *calls_data_path)

In [6]:
# путь папки с данными по распределению помощников врачей по врачам
present_assistants_path = r''
present_assistants_path = present_assistants_path.split('\\')
present_assistants_path = os.path.join(r'\\', *present_assistants_path)

In [7]:
# путь папки для кратковременных несистемных исключений
temporary_exception_path = r''
temporary_exception_path = temporary_exception_path.split('\\')
temporary_exception_path = os.path.join(r'\\', *temporary_exception_path)

In [8]:
# путь папки для кратковременных несистемных исключений
temporary_inclusion_path = r''
temporary_inclusion_path = temporary_inclusion_path.split('\\')
temporary_inclusion_path = os.path.join(r'\\', *temporary_inclusion_path)

In [152]:
# cловарь "номер_dat": "месяц", в который мы добавляем список месяцев, которые сейчас в работе

dat_month = dict()

# dat_month['may'] = '3196'
# dat_month['june'] = '3437'
# dat_month['july'] = '3582'
# dat_month['august'] = '3833'
dat_month['august_'] = '3940'

In [153]:
today = pd.Timestamp(dt.datetime.today().date())

today_ymd = today.strftime('%Y%m%d')
today_y_m_d = today.strftime('%Y_%m_%d')

yesterday = today - dt.timedelta(days=1)

yesterday_ymd = yesterday.strftime('%Y%m%d')
yesterday_y_m_d = yesterday.strftime('%Y-%m-%d')

In [154]:
# формируем даты, которые являются рабочими днями циклов

stages = dict()

stages[1] = pd.date_range('20220802', '20220810')
stages[2] = pd.date_range('20220811', '20220819')
stages[3] = pd.date_range('20220820', '20220831')

stages_and_days = dict()

for stage, dates in stages.items():
    
    stage_day_number = 0
    
    for weekday in [x.date() for x in dates if not x.day_name() in {'Saturday', 'Sunday'} ]:
        stages_and_days[weekday] = dict()
        stages_and_days[weekday]['stage'] = stage
        stage_day_number += 1
        stages_and_days[weekday]['stage_day'] = stage_day_number\
        
stages_df = pd.DataFrame(stages_and_days).T.reset_index()

In [155]:
column_names = {'Emiasid': 'emiasid', 'Emiasid ': 'emiasid' , 'emiasid ': 'emiasid', 'EMIAS_ID': 'emiasid', 'МО': 'mo', 
                'МУ': 'mu', 'Логин оператора': 'login', 'Дата и время звонка': 'call_date', 'Результат звонка': 'call_result',
                'Результат звонка ': 'call_result', 'Разговор состоялся': 'talk_result', 'Комменатрий': 'comment', 
                'Дата и время переноса': 'next_call_date', 'Контактный телефон': 'phone_number', 'ФИО врача': 'doctor_fio', 
                'ФИО помощника врача': 'assistant_fio', 'Логин помощника врача': 'login', 'Зал №': 'hall_number',
                'Логин naumen': 'login', 'МО закрепления': 'mo', 'Отсутствует на рабочем месте': 'absence'}

In [156]:
mo_list = {'ГП 64', 'ГП 191', 'ДКЦ 1', 'КДЦ 4', 'ДЦ 5', 'ГП 2', 'ГП 210', 'ГП 107', 'ГП 23', 'ГП 66', 'ДЦ 3', 'ГП 170', 
           'ГП 22', 'КДП 121', 'КДЦ 6', 'ГП 214', 'ГП 218', 'ГП 109', 'ГП 134', 'ГП 36', 'ГП 6', 'ГП 220', 'ГП 11'}

In [160]:
patient_current_statuses_path = rf'august__patients_current_statuses_{today_y_m_d}.xlsx'

# temporary_inclusion_path = 'можно_включать_в_обзвон_5_августа.xlsx'

calls_data_path = 'calls_data_20220802_20220815.xlsx'

assistant_to_doctor_distribution_path = 'Основное Раскрепление Август.xlsx'

# assistants_presence_path = 'Список ПВ для формирования задания.xlsx'

In [161]:
%%time

patients_current_statuses_reading = pd.read_excel(patient_current_statuses_path)


# temporary_inclusion_reading = pd.read_excel(temporary_inclusion_path, usecols=['EMIAS_ID'])


calls_data_reading = pd.read_excel(calls_data_path)


assistant_to_doctor_distribution_reading = pd.read_excel(assistant_to_doctor_distribution_path)

# assistants_presence_reading = pd.read_excel(assistants_presence_path)

# all_done_reading = pd.read_excel('all_done.xlsx')


second_stage_patients_reading = pd.read_excel('second_stage_patients.xlsx')

CPU times: total: 23.2 s
Wall time: 23.3 s


In [19]:
##  23.2 s

In [20]:
# как только получаем список ПВ на смене, перезапускаем эту ячейку и еще две, проверяем правильное количество отсуствующих ПВ
# и идем вниз

In [162]:
patients_current_statuses = \
patients_current_statuses_reading[['emiasid', 'date', 'status', 'old_comment', 'next_call_date',
       'doctor_fio', 'mo']].copy()


# temporary_inclusion = temporary_inclusion_reading.rename(columns=column_names).copy()


calls_data = calls_data_reading.rename(columns=column_names).copy()


assistant_to_doctor_distribution = assistant_to_doctor_distribution_reading.rename(columns=column_names)[['mo', 'doctor_fio', 
'assistant_fio', 'login']].copy()

second_stage_patients = second_stage_patients_reading.copy()

# all_done = all_done_reading.copy()

In [163]:
second_stage_calls_amount = \
second_stage_patients.groupby(by='emiasid').agg({'date': 'count'}).reset_index().sort_values(by='date', ascending=False)

In [164]:
second_stage_patients

,emiasid,date,status,old_comment,next_call_date,mo,login
0,18498110,2022-08-11,Не дозвонились,NaN,NaT,ГП 6,pomoshnik_pdn96
1,10468714,2022-08-11,Не дозвонились,NaN,NaT,ГП 6,pomoshnik_pdn96
2,20957244,2022-08-11,Не дозвонились,NaN,NaT,ГП 6,pomoshnik_pdn96
3,10127192,2022-08-11,Не дозвонились,NaN,NaT,ГП 6,pomoshnik_pdn96
4,24406315,2022-08-11,Не дозвонились,NaN,NaT,ГП 6,pomoshnik_pdn96
...,...,...,...,...,...,...,...
18471,28683589,2022-08-15,Не дозвонились,NaN,NaT,ГП 220,pomoshnik_pdn35
18472,30140586,2022-08-15,Не дозвонились,NaN,NaT,ГП 220,pomoshnik_pdn35
18473,30784142,2022-08-15,Не дозвонились,NaN,NaT,ГП 220,pomoshnik_pdn35
18474,28282873,2022-08-15,Не дозвонились,NaN,NaT,ГП 220,pomoshnik_pdn35


In [165]:
doctors_list = set(assistant_to_doctor_distribution.doctor_fio.unique())
doctors_list.add(np.NaN)

In [166]:
patients_current_statuses['emiasid'] = patients_current_statuses['emiasid'].astype(str).str.replace('\.0', '')
# temporary_inclusion['emiasid'] = temporary_inclusion['emiasid'].astype(str).str.replace('\.0', '')
calls_data['emiasid'] = calls_data['emiasid'].astype(str).str.replace('\.0', '')
second_stage_patients['emiasid'] = second_stage_patients['emiasid'].astype(str).str.replace('\.0', '')

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_4772\3427605406.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  patients_current_statuses['emiasid'] = patients_current_statuses['emiasid'].astype(str).str.replace('\.0', '')
C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_4772\3427605406.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  calls_data['emiasid'] = calls_data['emiasid'].astype(str).str.replace('\.0', '')
C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_4772\3427605406.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  second_stage_patients['emiasid'] = second_stage_patients['emiasid'].astype(str).str.replace('\.0', '')


In [167]:
patients_current_statuses['date'] = patients_current_statuses['date'].apply(lambda x: pd.to_datetime(x).date())
patients_current_statuses['next_call_date'] = patients_current_statuses['next_call_date'] \
.apply(lambda x: pd.to_datetime(x).date())

In [168]:
patients_current_statuses = patients_current_statuses[~patients_current_statuses.doctor_fio.isna()]

In [169]:
assistant_to_doctor_distribution.login.nunique()

141

In [170]:
actual_patients_for_task = patients_current_statuses

In [ ]:
# actual_patients_for_task = \
# actual_patients_for_task[
    
#     (
#         (actual_patients_for_task.mo.isin(still_big_task_mo_list)) & 
#      (actual_patients_for_task.status == 'Ни разу не звонили'))
     
#     |
    
#     ((~actual_patients_for_task.mo.isin(still_big_task_mo_list)) & (actual_patients_for_task.status.isin({
#         'Не дозвонились', 'Госпитализирован', 'Отказ', 'Другой врач в этой МО', 'Не нравится МО', 'Не нравится врач'})) 
#      & (~actual_patients_for_task.emiasid.isin(second_stage_patients.emiasid)))
# ]

In [172]:
first_stage_patients_for_task = \
actual_patients_for_task[actual_patients_for_task.status == 'Ни разу не звонили']
     

In [173]:
first_stage_patients_for_task = first_stage_patients_for_task.reset_index(drop=True)
first_stage_patients_for_task['priority'] = '1'

In [174]:
second_stage_patients_for_task = actual_patients_for_task[
    (actual_patients_for_task.status.isin(
        {'Не дозвонились', 'Госпитализирован', 'Отказ', 'Другой врач в этой МО', 'Не нравится МО', 'Не нравится врач'}))
     &
    (~actual_patients_for_task.emiasid.isin(second_stage_patients.emiasid))]

In [176]:
second_stage_patients_for_task = second_stage_patients_for_task.reset_index(drop=True)
second_stage_patients_for_task['priority'] = '4'

In [177]:
# actual_patients_for_task = actual_patients_for_task.reset_index(drop=True)
# actual_patients_for_task['priority'] = '3'

In [178]:
if today.day_name() == 'Monday':
    
    last_friday = (today - dt.timedelta(3)).date()
    
    call_back_today_patients = patients_current_statuses[
    (patients_current_statuses.status == 'Перезвонить сегодня') & 
    (patients_current_statuses.next_call_date == last_friday)]  
    
    call_back_on_date_patients = patients_current_statuses[
    (patients_current_statuses.status == 'Перенос звонка на другой день') 
    & patients_current_statuses.next_call_date.isin(
    [x.date() for x in pd.date_range((today - dt.timedelta(2)).date(), today)])]
    
else:
    
    call_back_today_patients = patients_current_statuses[
    (patients_current_statuses.status == 'Перезвонить сегодня') & 
    (patients_current_statuses.next_call_date == yesterday)]
    
    call_back_on_date_patients = patients_current_statuses[
    (patients_current_statuses.status == 'Перенос звонка на другой день') & 
    (patients_current_statuses.next_call_date == today)]
    

C:\Users\murad_satabaev\anaconda3\envs\pdn\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


In [179]:
call_back_today_patients = call_back_today_patients.reset_index(drop=True)
call_back_today_patients['priority'] = '2' 

In [180]:
call_back_today_patients.shape[0]

0

In [181]:
call_back_on_date_patients = call_back_on_date_patients.reset_index(drop=True)
call_back_on_date_patients['priority'] = '3'

In [182]:
call_back_on_date_patients.shape[0]

0

In [183]:
still_big_task_mo_list = {'ГП 11', 'ГП 210', 'КДЦ 4'}

In [184]:
final_actual_patients_for_task = pd.concat([
    first_stage_patients_for_task,
    call_back_today_patients, 
    call_back_on_date_patients,
    second_stage_patients_for_task]) \
.sort_values(by='priority').reset_index(drop=True)

In [185]:
final_actual_patients_for_task.shape[0]

1404

In [186]:
final_actual_patients_for_task.status.value_counts()

Ни разу не звонили    1403
Не дозвонились           1
Name: status, dtype: int64

In [187]:
final_actual_patients_for_task = final_actual_patients_for_task[['emiasid', 'date', 'status', 'old_comment', 'next_call_date',
       'doctor_fio', 'mo', 'priority']]

In [188]:
initial_assistant_to_patient_distribution_in_today_task = \
final_actual_patients_for_task.merge(assistant_to_doctor_distribution, on=['doctor_fio', 'mo']).sort_values(by='priority')

In [189]:
initial_assistant_to_patient_distribution_in_today_task.shape[0]

961

In [190]:
%%time

initial_task_for_today = pd.DataFrame()
for login in set(initial_assistant_to_patient_distribution_in_today_task.login):
    initial_assistant_to_patient_distribution_in_today_task_for_login = \
    initial_assistant_to_patient_distribution_in_today_task[initial_assistant_to_patient_distribution_in_today_task.login
         == login]
    call_back_number_for_login = initial_assistant_to_patient_distribution_in_today_task_for_login[
        initial_assistant_to_patient_distribution_in_today_task_for_login.status.isin({
            'Перезвонить сегодня', 'Перенос звонка на другой день'})].shape[0]

    initial_task_for_today = pd.concat(
        [initial_task_for_today, 

         initial_assistant_to_patient_distribution_in_today_task[
             (initial_assistant_to_patient_distribution_in_today_task.login==login) 
             & (initial_assistant_to_patient_distribution_in_today_task.mo.isin(still_big_task_mo_list))
         ].sort_values('priority').head((85 + call_back_number_for_login)),
         
         initial_assistant_to_patient_distribution_in_today_task[
             (initial_assistant_to_patient_distribution_in_today_task.login==login) &
             (~initial_assistant_to_patient_distribution_in_today_task.mo.isin(still_big_task_mo_list))
         ].sort_values('priority').head(85),
        
       
        ]).reset_index(drop=True)

CPU times: total: 188 ms
Wall time: 184 ms


In [191]:
initial_task_for_today[~initial_task_for_today.mo.isin(still_big_task_mo_list)] \
.groupby(['mo', 'login']).agg({'emiasid': 'count'}).sort_values(['mo', 'emiasid'], ascending=False).head(40)

emiasid
mo     login                    
ДЦ 5   pomoshnik_pdn88         1
ДКЦ 1  pomoshnik_pdn76        29
       pomoshnik_pdn95        28
       pomoshnik_pdn39        27
       pomoshnik_pdn36        21
       pomoshnik_pdn74        17
       pomoshnik_pdn78        17
       pomoshnik_pdn38        13
       pomoshnik_pdn77        12
ГП 23  pomoshnik_pdn161        1
ГП 220 pomoshnik_pdn73        25
       pomoshnik_pdn35        20
ГП 22  pomoshnik_pdn66        37
       pomoshnik_pdn203       32
       pomoshnik_pdn59        26
       pomoshnik_pdn91        19
       pomoshnik_pdn64        13
       pomoshnik_pdn61         9
ГП 218 pomoshnik_pdn8         38
       pomoshnik_pdn31        24
       pomoshnik_pdn29        17
       pomoshnik_pdn26        16
       pomoshnik_pdn27        16
       pomoshnik_pdn68        14
       pomoshnik_pdn69        13
       pomoshnik_pdn9         13
       pomoshnik_pdn67         7
       pomoshnik_pdn72         7
       pomoshnik_pdn70         6
ГП 191 pomoshnik_pdn15        43
       pomoshnik_pdn13        32
       pomoshnik_pdn20        32
       pomoshnik_pdn41        23
       pomoshnik_pdn222       16
       Помощник 3             13
ГП 134 pomoshnik_pdn89        52
       pomoshnik_pdn63        36
       pomoshnik_pdn75        30
       pomoshnik_pdn7         21
       pomoshnik_pdn93        20

In [192]:
initial_task_for_today[initial_task_for_today.mo.isin(still_big_task_mo_list)] \
.groupby(['mo', 'login']).agg({'emiasid': 'count'}).sort_values(['mo', 'emiasid'], ascending=False).head(10)

emiasid
mo    login                   
КДЦ 4 pomoshnik_pdn71       53
      pomoshnik_pdn58       28
      pomoshnik_pdn24       27
      pomoshnik_pdn2        17

In [193]:
# %%time
# initial_task_for_today = pd.DataFrame()
# for login in set(initial_assistant_to_patient_distribution_in_today_task.login):
#     initial_task_for_today = pd.concat([initial_task_for_today, 
#     initial_assistant_to_patient_distribution_in_today_task[initial_assistant_to_patient_distribution_in_today_task.login
#                                                             == login].sort_values('priority').head(85)]).reset_index(drop=True)

In [194]:
# initial_assistant_to_patient_distribution_in_today_task.groupby(['mo', 'login']).agg({'emiasid': 'count'}) \
# .sort_values(['mo', 'emiasid'], ascending=False).to_excel('оставшиеся пациенты.xlsx')

In [195]:
# 266 ms

In [196]:
# До этого момента можно делать без наличия списка отсуствующих ПВ

In [197]:
new_doctor_patients = final_actual_patients_for_task[(final_actual_patients_for_task.mo.isin(mo_list)) & 
                    (~final_actual_patients_for_task.doctor_fio.isin(doctors_list))].reset_index(drop=True)
new_doctor_patients = new_doctor_patients[~new_doctor_patients.emiasid.isin(calls_data.emiasid)]

In [198]:
initial_assistant_to_patient_distribution_in_today_task.shape[0] + new_doctor_patients \
.shape[0] + final_actual_patients_for_task[final_actual_patients_for_task.doctor_fio.isna()].shape[0]

1404

In [199]:
import gspread
from gspread_dataframe import get_as_dataframe
gc = gspread.service_account(filename='google_sheets_credentials.json') 
url = 'https://docs.google.com/spreadsheets/d/1TXjbk9tRDMCjEyyiuK5OIgT8xwlHQ2V1PX2zaVnxPXY/edit#gid=0'
sh = gc.open_by_url(url)
ws = sh.get_worksheet(0)
def search_row(ws, desired_value):
    row_number = 0
    for row in ws.get_all_values():
        row_number += 1
        for value in row:
            if value == desired_value:
                return row_number
            
skiprows = search_row(ws, 'ФИО помощника врача') - 1

assistants_presence_reading = get_as_dataframe(ws, evaluate_formulas=True, skiprows=skiprows)
assistants_presence_reading = assistants_presence_reading.loc[~(assistants_presence_reading['Зал №'] == 'Зал №'), 
    ['Зал №', 'МО закрепления', 'ФИО помощника врача', 'Логин naumen', 'Отсутствует на рабочем месте']] \
        .dropna(how='all', axis=0)


assistants_presence = assistants_presence_reading.rename(columns=column_names).copy()

absent_assistants = assistants_presence[assistants_presence.absence == 1]

absent_assistants.shape[0]

44

In [200]:
initial_task_for_today_present_assistants = initial_task_for_today[~initial_task_for_today.login.isin(absent_assistants.login)]

In [201]:
initial_task_for_today_present_assistants.groupby('login').agg({'emiasid': 'count'}).sort_values('emiasid', ascending=False)

,emiasid
login,
pomoshnik_pdn71,53
pomoshnik_pdn89,52
pomoshnik_pdn15,43
pomoshnik_pdn8,38
pomoshnik_pdn203,32
pomoshnik_pdn75,30
pomoshnik_pdn76,29
pomoshnik_pdn95,28
pomoshnik_pdn39,27


In [202]:
initial_task_for_today_absent_assistants =  initial_task_for_today[initial_task_for_today.login.isin(absent_assistants.login)] \
[['emiasid', 'date', 'status', 'old_comment', 'next_call_date', 'doctor_fio', 'mo', 'priority']]

In [203]:
additional_task_for_present_assistants = \
pd.concat([
    new_doctor_patients,
    initial_task_for_today_absent_assistants
]).sort_values('priority').reset_index(drop=True)

In [204]:
additional_task_for_present_assistants_by_mo = \
additional_task_for_present_assistants.groupby(['mo']).agg({'emiasid': 'count'}).sort_values(by=['mo', 'emiasid'])

In [205]:
distribution_count = initial_task_for_today_present_assistants.groupby(['mo']).agg({'emiasid': 'count', 'login': 'nunique'}) \
.sort_values(by=['mo', 'emiasid'])

In [206]:
distribution_count['total_delta'] = distribution_count['login'] * 85 - distribution_count['emiasid']

In [207]:
distribution_count = distribution_count[distribution_count.total_delta > 0]

In [208]:
final_task_predistribution = \
pd.concat([initial_task_for_today_present_assistants, additional_task_for_present_assistants]).reset_index(drop=True)

In [209]:
empty_login_patients_amount_before = final_task_predistribution[final_task_predistribution.login.isna()].shape[0]
empty_login_patients_amount_before

721

In [211]:
%%time

for mo in distribution_count.index:
    
    while final_task_predistribution[final_task_predistribution.mo == mo].groupby('login') \
    .agg({'emiasid': 'count'}).min()['emiasid'] < 85 and \
    final_task_predistribution[(final_task_predistribution.login.isna()) & (final_task_predistribution.mo == mo)] \
    .shape[0] > 0:
        
        miminal_in_mo_assistant_emiasid_amount = final_task_predistribution[final_task_predistribution.mo == mo] \
        .groupby('login').agg({'emiasid': 'count'}).min()['emiasid']

        minimal_in_mo_emiasid_amount_login = final_task_predistribution[final_task_predistribution.mo == mo] \
        .groupby('login').agg({'emiasid': 'count'}).sort_values(by='emiasid').index[0]
        
        
        empty_login_in_mo_index = final_task_predistribution[
            (final_task_predistribution.login.isna()) & (final_task_predistribution.mo == mo)
        ].sort_values(by='priority').index[0]

        final_task_predistribution.loc[empty_login_in_mo_index, 'login'] = minimal_in_mo_emiasid_amount_login


CPU times: total: 31.2 ms
Wall time: 34.1 ms


In [212]:
empty_login_patients_amount_after = final_task_predistribution[final_task_predistribution.login.isna()].shape[0]
empty_login_patients_amount_after

95

In [213]:
check_table = distribution_count.merge(additional_task_for_present_assistants_by_mo,left_index=True, right_index=True) \
[['total_delta', 'emiasid_y']]

In [214]:
check_table.loc[check_table['total_delta'] > check_table['emiasid_y'], 'real_addition'] = \
check_table.loc[check_table['total_delta'] > check_table['emiasid_y'], 'emiasid_y']

check_table.loc[check_table['total_delta'] <= check_table['emiasid_y'], 'real_addition'] = \
check_table.loc[check_table['total_delta'] <= check_table['emiasid_y'], 'total_delta']

In [215]:
real_change_of_emty_login_patients_amount = check_table.real_addition.sum()
real_change_of_emty_login_patients_amount

626.0

In [216]:
empty_login_patients_amount_before - empty_login_patients_amount_after == real_change_of_emty_login_patients_amount

True

In [217]:
final_task_distribution = final_task_predistribution[~final_task_predistribution.login.isna()] \
.copy().drop(columns={'doctor_fio', 'assistant_fio'}).reset_index(drop=True)

In [219]:
final_task_distribution

,emiasid,date,status,old_comment,next_call_date,mo,priority,login
0,20358234,2022-08-16,Ни разу не звонили,NaN,NaT,ГП 218,1,pomoshnik_pdn31
1,274989991,2022-08-16,Ни разу не звонили,NaN,NaT,ГП 218,1,pomoshnik_pdn31
2,2245966334,2022-08-16,Ни разу не звонили,NaN,NaT,ГП 218,1,pomoshnik_pdn31
3,2245665810,2022-08-16,Ни разу не звонили,NaN,NaT,ГП 218,1,pomoshnik_pdn31
4,25470391,2022-08-16,Ни разу не звонили,NaN,NaT,ГП 218,1,pomoshnik_pdn31
...,...,...,...,...,...,...,...,...
1304,16575457,2022-08-16,Ни разу не звонили,NaN,NaT,ДКЦ 1,1,pomoshnik_pdn74
1305,16764281,2022-08-16,Ни разу не звонили,NaN,NaT,ГП 134,1,pomoshnik_pdn89
1306,16735916,2022-08-16,Ни разу не звонили,NaN,NaT,ДКЦ 1,1,pomoshnik_pdn76
1307,17620820,2022-08-16,Ни разу не звонили,NaN,NaT,ГП 22,1,pomoshnik_pdn64


In [220]:
# final_task_amount_for_assistant = final_task_distribution.groupby('login').agg({'emiasid': 'count'}).reset_index()

In [221]:
# patients_data_for_additional_task_for_low_occupied_assistants = \
# patients_current_statuses.merge(assistant_to_doctor_distribution, on=['doctor_fio', 'mo'])
# patients_data_for_additional_task_for_low_occupied_assistants = patients_data_for_additional_task_for_low_occupied_assistants[
# patients_data_for_additional_task_for_low_occupied_assistants.status.isin({'Разговор состоялся', 'Запишется самостоятельно'})]

In [121]:
# low_occupied_assistants = set(final_task_amount_for_assistant[final_task_amount_for_assistant.emiasid < 50].login.unique())
# no_task_assistants = set(assistants_presence[(~assistants_presence.login.isin(final_task_amount_for_assistant.login)) &
#                        (assistants_presence.absence.isna())].login.unique())


In [122]:
# additional_task_for_low_occupied_assistants = patients_data_for_additional_task_for_low_occupied_assistants[
#     patients_data_for_additional_task_for_low_occupied_assistants.login.isin((low_occupied_assistants |  no_task_assistants))] \
#     .drop(columns={'doctor_fio', 'assistant_fio'}).reset_index(drop=True)

In [123]:
# additional_task_for_low_occupied_assistants['date'] = additional_task_for_low_occupied_assistants['date'] \
# .apply(lambda x: pd.to_datetime(x).date())
# additional_task_for_low_occupied_assistants['next_call_date'] = additional_task_for_low_occupied_assistants['next_call_date'] \
# .apply(lambda x: pd.to_datetime(x).date())

In [124]:
# additional_task_for_low_occupied_assistants['priority'] = '4'

In [125]:
# final_task_distribution_with_additional_task_for_low_occupied_assistants = \
# pd.concat([final_task_distribution, additional_task_for_low_occupied_assistants])

In [126]:
# final_task_distribution_with_additional_task_for_low_occupied_assistants.shape[0]

In [127]:
# final_task_distribution_with_additional_task_for_low_occupied_assistants \
# .groupby('login').agg({'emiasid': 'count'}).sort_values('emiasid', ascending=False).head(30)

In [128]:
# %%time

# final_task_distribution_with_additional_task_for_low_occupied_assistants_for_today = pd.DataFrame()
# for login in set(final_task_distribution_with_additional_task_for_low_occupied_assistants.login):
#     final_task_distribution_with_additional_task_for_low_occupied_assistants_for_today = pd.concat(
#         [final_task_distribution_with_additional_task_for_low_occupied_assistants_for_today,
#          final_task_distribution_with_additional_task_for_low_occupied_assistants[
#              final_task_distribution_with_additional_task_for_low_occupied_assistants.login
#                                                             == login].sort_values('priority').head(85)]).reset_index(drop=True)

In [129]:
# final_task_distribution_with_additional_task_for_low_occupied_assistants_for_today \
# .groupby('login').agg({'emiasid': 'count'}).sort_values('emiasid', ascending=False)

# ПРОВЕРКА

In [222]:
assistants_presence[assistants_presence.absence == 1].shape[0]

44

In [223]:
empty_login_patients_amount_before

721

In [224]:
empty_login_patients_amount_after

95

In [225]:
real_change_of_emty_login_patients_amount

626.0

In [226]:
empty_login_patients_amount_before - empty_login_patients_amount_after == real_change_of_emty_login_patients_amount

True

In [227]:
final_task_distribution.shape[0] + empty_login_patients_amount_after

1404

In [228]:
initial_task_for_today.shape[0] + new_doctor_patients.shape[0]

1404

In [229]:
final_task_distribution.emiasid.isin(
    patients_current_statuses.emiasid).unique()

array([ True])

In [230]:
# ~final_task_distribution.emiasid.isin(
#     temporary_inclusion.emiasid).unique()

In [231]:
final_task_distribution[final_task_distribution.status == 'Ни разу не звонили'].emiasid.isin(calls_data.emiasid).unique()

array([False])

In [232]:
final_task_distribution[~(final_task_distribution.status == 'Ни разу не звонили')].emiasid.isin(calls_data.emiasid).unique()

array([ True])

In [233]:
call_back_today_patients.shape[0]

0

In [234]:
final_task_distribution[final_task_distribution.status == 'Перезвонить сегодня'].shape[0]

0

In [235]:
call_back_on_date_patients.shape[0]

0

In [236]:
final_task_distribution[final_task_distribution.status == 'Перенос звонка на другой день'].shape[0]

0

In [237]:
final_task_distribution.shape[0]

1309

In [238]:
# additional_task_for_low_occupied_assistants.shape[0]

In [239]:
# additional_task_for_low_occupied_assistants.columns

In [240]:
# additional_task_for_low_occupied_assistants.groupby('login').agg({'emiasid': 'count'}).sort_values('emiasid', ascending=False)

In [241]:
final_task_distribution.shape[0]

1309

In [150]:
# final_task_distribution_with_additional_task_for_low_occupied_assistants_for_today.shape[0]

# ЭКСПОРТ

In [242]:
final_task_distribution.to_excel(rf'august__final_task_distribution_{today_y_m_d}.xlsx', index=False)

In [ ]:
# additional_task_for_low_occupied_assistants.to_excel('august_additional_task_for_pdn39.xlsx', index=False)

In [ ]:
# final_task_distribution_with_additional_task_for_low_occupied_assistants_for_today \
# .to_excel(rf'august_final_task_distribution_{today_y_m_d}.xlsx', index=False)

# МАНИПУЛЯЦИИ ВНЕ ФОРМИРОВАНИЯ ЗАДАНИЯ

In [72]:
present_assistants = assistants_presence[assistants_presence.absence.isna()]

In [73]:
present_assistants[~present_assistants.login.isin(final_task_distribution.login)]

,hall_number,mo,assistant_fio,login,absence
3,509,ГП 22,Катько Алексей Алексеевич,pomoshnik_pdn61,NaN
36,507,ГП 22,Рамазанова Джульета Руслановна,pomoshnik_pdn64,NaN
79,504,КДЦ 6,Капустина Ольга Владимировна,pomoshnik_pdn37,NaN
172,504,КДЦ 6,Капустина Ольга Владимировна,pomoshnik_pdn37,NaN


In [ ]:
~final_task_distribution.login.isin(assistants_presence[assistants_presence.absence.isna()].login)

In [ ]:
final_task_distribution[final_task_distribution.login]

In [ ]:
df = assistants_presence.sort_values(['mo', 'login'])

In [ ]:
df[df.mo == 'ГП 218']

In [ ]:
additional_task_for_present_assistants_by_mo

In [ ]:
check_table

In [ ]:
patients_current_statuses[
    (patients_current_statuses.mo == 'ГП 218') & (patients_current_statuses.status == 'Ни разу не звонили')].shape[0]

In [ ]:
initial_task_for_today[initial_task_for_today.login == 'pomoshnik_pdn39']

In [ ]:
initial_assistant_to_patient_distribution_in_today_task[
    initial_assistant_to_patient_distribution_in_today_task.login == 'pomoshnik_pdn130']

In [ ]:
df_1 = patients_current_statuses.merge(assistant_to_doctor_distribution, on=['doctor_fio', 'mo'])

In [ ]:
df_1.groupby(['mo', 'login']).agg({'emiasid': 'count'}).sort_values(['mo', 'emiasid'], ascending=False).to_excel(
'изначальное распределение пациентов по помощникам.xlsx')

In [ ]:
df_1[df_1.login == 'pomoshnik_pdn70'].status.value_counts()

In [ ]:
(assistant_to_doctor_distribution[
    ~assistant_to_doctor_distribution.login.isin(
    initial_assistant_to_patient_distribution_in_today_task.login)])[['login']].unique()

# ПОИСК ПВ У КОТОРЫХ ЗАКАНЧИВАЕТСЯ ЗАДАНИЕ

In [ ]:
df_1 = patients_current_statuses.merge(assistant_to_doctor_distribution, on=['doctor_fio', 'mo'])

In [ ]:
df_1.to_excel('emiasid_login_status.xlsx', index=False)

In [ ]:
df_2 = df_1[df_1.status == 'Ни разу не звонили']

In [ ]:
df_3 = df_2.groupby('login').agg({'emiasid': 'count'}).reset_index()

In [ ]:
df_4 = df_3[df_3.emiasid < 86]

In [ ]:
df_4.shape[0]

In [ ]:
df_5 = final_task_distribution_with_additional_task_for_low_occupied_assistants_for_today \
[final_task_distribution_with_additional_task_for_low_occupied_assistants_for_today.status == 'Ни разу не звонили']\
.groupby('login').agg({'emiasid': 'count'}).reset_index()

In [ ]:
df_6 = df_4.merge(df_5, on='login')

In [ ]:
df_6['delta'] = df_6.emiasid_x - df_6.emiasid_y

In [ ]:
df_6[df_6.delta <= 0][['login']].shape[0]

In [ ]:
df_6[df_6.delta <= 0][['login']].to_excel('логины помощников, у которых сегодня закончится задание.xlsx', index=False)

In [ ]:
data_1 = final_task_predistribution.groupby(['mo', 'login']).agg({'emiasid': 'count'}).sort_values(['mo', 'emiasid'], 
                                                                                                   ascending=False)

In [ ]:
data_1[data_1.emiasid < 80].reset_index()[['login']].to_excel('пв с маленьким заданием.xlsx', index=False)

In [ ]:
import pandas as pd

In [ ]:
fs = pd.read_excel('first_stage_august_final_task_distribution_2022_08_12.xlsx')

In [ ]:
ss = pd.read_excel('second_stage_august_final_task_distribution_2022_08_12.xlsx')

In [ ]:
common_file = pd.concat([fs, ss])

In [ ]:
common_file = common_file.sort_values(by='priority')

In [ ]:
common_file.to_excel(rf'august_final_task_distribution_{today_y_m_d}.xlsx', index=False)

In [ ]:
common_file

In [ ]:
still_big_task_mo_list = {'ГП 109', 'ГП 11', 'ГП 210', 'ДЦ 3', 'КДЦ 4'}

In [ ]:
still_big_task_mo_list

In [ ]:
final_task = pd.DataFrame()
for login in set(common_file.login):
    final_task = pd.concat([
        final_task, 
                            
                           common_file[
                               (common_file.login == login) 
                               &
                               (~common_file.mo.isin(still_big_task_mo_list))
                           ].sort_values(by='priority').head(85),
                           
                           common_file[
                               (common_file.login == login) 
                               &
                               (common_file.mo.isin(still_big_task_mo_list))
                           ].sort_values(by='priority')                         
                          
                           ]
                          
                          )
    

In [ ]:
%%time

initial_task_for_today = pd.DataFrame()
for login in set(initial_assistant_to_patient_distribution_in_today_task.login):


    initial_task_for_today = pd.concat(
        [initial_task_for_today, 

#          initial_assistant_to_patient_distribution_in_today_task[
#              (initial_assistant_to_patient_distribution_in_today_task.login==login) 
#              & (initial_assistant_to_patient_distribution_in_today_task.mo.isin(still_big_task_mo_list))
#          ].sort_values('priority').head((85 + call_back_number_for_login)),
         
         initial_assistant_to_patient_distribution_in_today_task[
             (initial_assistant_to_patient_distribution_in_today_task.login==login) &
             (~initial_assistant_to_patient_distribution_in_today_task.mo.isin(still_big_task_mo_list))
         ].sort_values('priority').head(85),
        
       
        ]).reset_index(drop=True)

In [ ]:
common_file.shape[0]

In [ ]:
final_task.shape[0]

In [ ]:
final_task.to_excel(rf'august_final_task_distribution.xlsx', index=False)

In [ ]:
f = pd.read_excel('august_final_task_distribution.xlsx')

In [ ]:
f.columns

In [ ]:
f['emiasid'] = f['emiasid'].astype(str).str.replace('\.0', '')

In [ ]:
f.to_excel(rf'august_final_task_distribution.xlsx', index=False)